In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib

You should consider upgrading via the 'C:\Users\Andyj\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.



  Using cached pandas-1.5.3-cp39-cp39-win_amd64.whl (10.9 MB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)


You should consider upgrading via the 'C:\Users\Andyj\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Using cached matplotlib-3.7.1-cp39-cp39-win_amd64.whl (7.6 MB)
  Using cached kiwisolver-1.4.4-cp39-cp39-win_amd64.whl (55 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached contourpy-1.0.7-cp39-cp39-win_amd64.whl (160 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
     ---------------------------------------- 1.0/1.0 MB 12.8 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\Andyj\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import io, time, json
import requests
from bs4 import BeautifulSoup

In [ ]:
'''
Reduce size of the city of los angeles crime data sets:
- Keep only features of interest: 
  + Crm Cd Desc (Nominal)
  + Premis Desc (Nominal)
  + Year (Nominal)
  
IMPORANT: the file 'Los-Angeles-Crime-2001-Present.csv' is not uploaded 
as the size is above github's file size restriction so running this cell
will clause an error
'''
df = pd.read_csv('Los-Angeles-Crime-2001-Present.csv')
# make a new column for year so we can drop 
# the date column while reducing data size
df['Year'] = pd.to_datetime(df['DATE OCC']).dt.year
# keep only columns of interest
df = df[['Year', 'Crm Cd Desc', 'Premis Desc']]
print(df.head())
df.to_csv('crime_los_angeles_reduced.csv')

In [ ]:
''' 
Reduce size of the chicago crime data sets:
- Keep only features of interest: 
  + Primary Type (Nominal)
  + Location Description (Nominal)
  + Domestic (Binary)
  + Year (Nominal)
- Shrink year range to 2020-2023 so we match up with los angeles data.

IMPORANT: the file 'Chicago-Crime-2001-Present.csv' is not uploaded 
as the size is above github's file size restriction so running this cell
will clause an error
'''
df = pd.read_csv('Chicago-Crime-2001-Present.csv')
# keep only columns of interest
df = df[['Year', 'Primary Type', 'Location Description', 'Domestic']]
# shrink date range
df = df[df['Year'] >= 2020]
print(df.head())
df.to_csv('crime_chicago_reduced.csv')

In [3]:
# check the data
chicago_crime_df = pd.read_csv('crime_chicago_reduced.csv')
print(chicago_crime_df.head())
los_crime_df = pd.read_csv('crime_los_angeles_reduced.csv')
print(los_crime_df.head())

   Unnamed: 0  Year         Primary Type Location Description  Domestic
0          90  2020                THEFT               STREET     False
1         420  2020  MOTOR VEHICLE THEFT            APARTMENT      True
2         812  2020              ASSAULT            RESIDENCE     False
3         855  2020   DECEPTIVE PRACTICE      OTHER (SPECIFY)     False
4        1348  2020            NARCOTICS             SIDEWALK     False
   Unnamed: 0  Year                                        Crm Cd Desc  \
0           0  2020                           BATTERY - SIMPLE ASSAULT   
1           1  2020                           BATTERY - SIMPLE ASSAULT   
2           2  2020          SEX OFFENDER REGISTRANT OUT OF COMPLIANCE   
3           3  2020           VANDALISM - MISDEAMEANOR ($399 OR UNDER)   
4           4  2020  VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...   

                                    Premis Desc  
0                        SINGLE FAMILY DWELLING  
1                      

In [8]:
# we see that our data has an Unnamed column from the indices in the pre-processing step
chicago_crime_df = chicago_crime_df.drop(columns=['Unnamed: 0'])
chicago_crime_df

,Year,Primary Type,Location Description,Domestic
0,2020,THEFT,STREET,False
1,2020,MOTOR VEHICLE THEFT,APARTMENT,True
2,2020,ASSAULT,RESIDENCE,False
3,2020,DECEPTIVE PRACTICE,OTHER (SPECIFY),False
4,2020,NARCOTICS,SIDEWALK,False
...,...,...,...,...
696504,2022,DECEPTIVE PRACTICE,NaN,False
696505,2022,CRIMINAL DAMAGE,STREET,False
696506,2022,MOTOR VEHICLE THEFT,PARKING LOT / GARAGE (NON RESIDENTIAL),False
696507,2022,BURGLARY,APARTMENT,False


In [12]:
# we also remove this column for los angeles data
los_crime_df = los_crime_df.drop(columns=['Unnamed: 0'])
# additionally, we see that the column names 'Crm Cd Desc' and 'Premis Desc'
# is confusing so we rename them to their equivalent
los_crime_df = los_crime_df.rename(columns={
    'Crm Cd Desc': 'Primary Type',
    'Premis Desc': 'Location Description'
})
los_crime_df

,Year,Primary Type,Location Description
0,2020,BATTERY - SIMPLE ASSAULT,SINGLE FAMILY DWELLING
1,2020,BATTERY - SIMPLE ASSAULT,SIDEWALK
2,2020,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,POLICE FACILITY
3,2020,VANDALISM - MISDEAMEANOR ($399 OR UNDER),"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
4,2020,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",BEAUTY SUPPLY STORE
...,...,...,...
682330,2023,THEFT OF IDENTITY,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)"
682331,2023,THEFT PLAIN - PETTY ($950 & UNDER),STREET
682332,2023,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",HEALTH SPA/GYM
682333,2023,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...","VEHICLE, PASSENGER/TRUCK"


In [13]:
def describe(df):
    '''
    Give some preliminary observation on the given dataframe:
    - Size and data type
    - NaN count each column
    - Value count each column
    '''
    print('**************************')
    print('*** Size and data type')
    size = len(df)
    print(f'Size: {df.size}')
    print(df.dtypes)
    print('**************************')
    print('*** NaN count each column')
    nan_df = df.isna().sum().to_frame(name='count')
    nan_df['percent'] = nan_df['count'] / size * 100
    print(nan_df)
    for col in df:
        print('**************************')
        print(f'\n*** Value count of {col}')
        val_df = df[col].value_counts().to_frame(name='count')
        val_df['percent'] = val_df['count'] / size * 100
        print(val_df)

In [14]:
describe(chicago_crime_df)

**************************

*** Size and data type
Size: 2786036
Year                     int64
Primary Type            object
Location Description    object
Domestic                  bool
dtype: object
**************************

*** NaN count each column
                      count   percent
Year                      0  0.000000
Primary Type              0  0.000000
Location Description   3061  0.439477
Domestic                  0  0.000000
**************************

*** Value count each column
**************************
       count    percent
2022  237221  34.058569
2020  212042  30.443541
2021  208445  29.927108
2023   38801   5.570782
**************************
                                    count    percent
THEFT                              144697  20.774606
BATTERY                            129070  18.530988
CRIMINAL DAMAGE                     81553  11.708822
ASSAULT                             62577   8.984378
DECEPTIVE PRACTICE                  53492   7.680016
MOTOR

In [16]:
# column Location Description has very few row with NaN value
# we can drop that
chicago_crime_df = chicago_crime_df[~chicago_crime_df['Location Description'].isna()]
describe(chicago_crime_df)

**************************

*** Size and data type
Size: 2773792
Year                     int64
Primary Type            object
Location Description    object
Domestic                  bool
dtype: object
**************************

*** NaN count each column
                      count  percent
Year                      0      0.0
Primary Type              0      0.0
Location Description      0      0.0
Domestic                  0      0.0
**************************

*** Value count each column
**************************
       count    percent
2022  236476  34.101476
2020  210743  30.390599
2021  207577  29.934040
2023   38652   5.573886
**************************
                                    count    percent
THEFT                              144694  20.865876
BATTERY                            129069  18.612643
CRIMINAL DAMAGE                     81553  11.760507
ASSAULT                             62577   9.024036
DECEPTIVE PRACTICE                  50437   7.273364
MOTOR VEHI

In [17]:
describe(los_crime_df)

**************************

*** Size and data type
Size: 2047005
Year                     int64
Primary Type            object
Location Description    object
dtype: object
**************************

*** NaN count each column
                      count  percent
Year                      0  0.00000
Primary Type              0  0.00000
Location Description    357  0.05232
**************************

*** Value count each column
**************************
       count    percent
2022  232194  34.029326
2021  208854  30.608719
2020  199022  29.167784
2023   42265   6.194171
**************************
                                                    count    percent
VEHICLE - STOLEN                                    73485  10.769637
BATTERY - SIMPLE ASSAULT                            53731   7.874578
THEFT OF IDENTITY                                   44602   6.536672
BURGLARY FROM VEHICLE                               42870   6.282838
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VAN... 

In [18]:
# column Location Description also has very few row with NaN value
# we can drop that
los_crime_df = los_crime_df[~los_crime_df['Location Description'].isna()]
describe(los_crime_df)

**************************

*** Size and data type
Size: 2045934
Year                     int64
Primary Type            object
Location Description    object
dtype: object
**************************

*** NaN count each column
                      count  percent
Year                      0      0.0
Primary Type              0      0.0
Location Description      0      0.0
**************************

*** Value count each column
**************************
       count    percent
2022  232048  34.025731
2021  208762  30.611251
2020  198954  29.173082
2023   42214   6.189936
**************************
                                                    count    percent
VEHICLE - STOLEN                                    73483  10.774981
BATTERY - SIMPLE ASSAULT                            53721   7.877234
THEFT OF IDENTITY                                   44601   6.539947
BURGLARY FROM VEHICLE                               42870   6.286127
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VAN... 